In [1]:
# import time
# import PySide6 #PyQt6
# time.clock = time.time
# import vispy
# vispy.__dir__()
# # vispy.test()
# %gui qt

# print(time)

# print(vispy.__dir__())
# # vispy.test()

In [30]:
## IMPORTS
import pydicom
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
import cv2
import datetime

import numpy as np
import cupy as cp

na                  = np.asarray
xm = {
    np:             (np.asarray, np.asarray, np.asarray, np),
    cp:             (cp.asarray, cp.asnumpy, cp.asarray, cp),
}  # (da, ha, xa, xp)
xdarray2xp          = lambda xdarray: {np.ndarray: np, cp.ndarray: cp}[type(xdarray)]

AXALL, AXNEW = (slice(None),), (None,)

ceil = lambda x: int(x) + (int(x) != x)

tdsp = r"../AngioCT tt szyjnych/1.3.12.2.1107.5.1.4.105397.30000021102806505454700023291/*"
# tdsp = r"../AngioCT tt szyjnych/1/AQWS01_10.7.112.230_20220531142257868/1.3.12.2.1107.5.1.4.60193.30000009040306315196800011793/*"
# tdsp = r"../AngioCT tt szyjnych/cab/*"
ds_path_l = glob.glob(tdsp)

ds_l = [pydicom.dcmread(ds_path) for ds_path in ds_path_l]
dsa  = na([ds.pixel_array for ds in ds_l[::-1]])

# for ds in ds_l: print(ds.AcquisitionTime)

# https://pydicom.github.io/pydicom/stable/auto_examples/image_processing/reslice.html#sphx-glr-auto-examples-image-processing-reslice-py
ds = ds_l[2]
ds.dir

<bound method Dataset.dir of Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 212
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 2.16.840.1.113669.632.21.1635916553.279322404.391980493922060650
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 2.16.840.1.113669.632.21.1
(0002, 0013) Implementation Version Name         SH: 'AQNET430-370IB5'
(0002, 0016) Source Application Entity Title     AE: ''
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID         

In [3]:
import sys

from cuda import cudart

import numpy as np
import cupy as cp

import pyrr
import glfw

from OpenGL.GL import *  # noqa F403
import OpenGL.GL.shaders


def format_cudart_err(err):
    return (
        f"{cudart.cudaGetErrorName(err)[1].decode('utf-8')}({int(err)}): "
        f"{cudart.cudaGetErrorString(err)[1].decode('utf-8')}"
    )


def check_cudart_err(args):
    if isinstance(args, tuple):
        assert len(args) >= 1
        err = args[0]
        if len(args) == 1:
            ret = None
        elif len(args) == 2:
            ret = args[1]
        else:
            ret = args[1:]
    else:
        err = args
        ret = None

    assert isinstance(err, cudart.cudaError_t), type(err)
    if err != cudart.cudaError_t.cudaSuccess:
        raise RuntimeError(format_cudart_err(err))

    return ret


class CudaOpenGLMappedBuffer:
    def __init__(self, gl_buffer, flags=0):
        self._gl_buffer = int(gl_buffer)
        self._flags = int(flags)

        self._graphics_ressource = None
        self._cuda_buffer = None

        self.register()

    @property
    def gl_buffer(self):
        return self._gl_buffer

    @property
    def cuda_buffer(self):
        assert self.mapped
        return self._cuda_buffer

    @property
    def graphics_ressource(self):
        assert self.registered
        return self._graphics_ressource

    @property
    def registered(self):
        return self._graphics_ressource is not None

    @property
    def mapped(self):
        return self._cuda_buffer is not None

    def __enter__(self):
        return self.map()

    def __exit__(self, exc_type, exc_value, trace):
        self.unmap()
        return False

    def __del__(self):
        self.unregister()

    def register(self):
        if self.registered:
            return self._graphics_ressource
        print(self._gl_buffer, type(self._gl_buffer))
        self._graphics_ressource = check_cudart_err(
            cudart.cudaGraphicsGLRegisterBuffer(self._gl_buffer, self._flags)
            # cudart.cudaGraphicsGLRegisterImage(self._gl_buffer, GL_TEXTURE_3D, self._flags )
        )
        return self._graphics_ressource

    def unregister(self):
        if not self.registered:
            return self
        self.unmap()
        self._graphics_ressource = check_cudart_err(
            cudart.cudaGraphicsUnregisterResource(self._graphics_ressource)
        )
        return self

    def map(self, stream=None):
        if not self.registered:
            raise RuntimeError("Cannot map an unregistered buffer.")
        if self.mapped:
            return self._cuda_buffer
        
        print(self._graphics_ressource)

        check_cudart_err(
            cudart.cudaGraphicsMapResources(1, self._graphics_ressource, stream)
        )

        ptr, size = check_cudart_err(
            cudart.cudaGraphicsResourceGetMappedPointer(self._graphics_ressource)
        )

        self._cuda_buffer = cp.cuda.MemoryPointer(
            cp.cuda.UnownedMemory(ptr, size, self), 0
        )

        return self._cuda_buffer

    def unmap(self, stream=None):
        if not self.registered:
            raise RuntimeError("Cannot unmap an unregistered buffer.")
        if not self.mapped:
            return self

        self._cuda_buffer = check_cudart_err(
            cudart.cudaGraphicsUnmapResources(1, self._graphics_ressource, stream)
        )

        return self


class CudaOpenGLMappedArray(CudaOpenGLMappedBuffer):
    def __init__(self, dtype, shape, gl_buffer, flags=0, strides=None, order='C'):
        super().__init__(gl_buffer, flags)
        self._dtype = dtype
        self._shape = shape
        self._strides = strides
        self._order = order

    @property
    def cuda_array(self):
        assert self.mapped
        return cp.ndarray(
            shape=self._shape,
            dtype=self._dtype,
            strides=self._strides,
            order=self._order,
            memptr=self._cuda_buffer,
        )

    def map(self, *args, **kwargs):
        super().map(*args, **kwargs)
        return self.cuda_array


VERTEX_SHADER = """
#version 330
in vec3 position;
uniform mat4 transform;
void main() {
    gl_Position = transform * vec4(position, 1.0f);
}
"""


FRAGMENT_SHADER = """
#version 330
out vec4 outColor;
void main() {
    outColor = vec4(1.0f, 0.0f, 0.0f, 1.0f);
}
"""


def setup_buffers(Nx, Ny):
    max_vertices = Nx*Ny
    max_triangles = 2*(Nx-1)*(Ny-1)

    ftype = np.float32
    itype = np.uint32

    vertex_bytes = 3 * max_vertices * ftype().nbytes
    index_bytes = 3 * max_triangles * itype().nbytes

    flags = cudart.cudaGraphicsRegisterFlags.cudaGraphicsRegisterFlagsWriteDiscard

    VBO = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, VBO)
    glBufferData(GL_ARRAY_BUFFER, vertex_bytes, None, GL_DYNAMIC_DRAW)
    vertex_buffer = CudaOpenGLMappedArray(ftype, (Ny, Nx, 3), VBO, flags)
    # vertex_buffer = CudaOpenGLMappedArray(ftype, (Ny, Nx, 3), 2, flags)

    EBO = glGenBuffers(1)
    glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, EBO)
    glBufferData(GL_ELEMENT_ARRAY_BUFFER, index_bytes, None, GL_DYNAMIC_DRAW)
    index_buffer = CudaOpenGLMappedArray(itype, (Ny-1, Nx-1, 2, 3), EBO, flags)

    Ix = cp.arange(Nx-1)[None,:]
    Iy = cp.arange(Ny-1)[:,None]

    I = index_buffer.map()

    I[...,0,0] = Iy*Nx+Ix
    I[...,0,1] = Iy*Nx+Ix+Nx
    I[...,0,2] = Iy*Nx+Ix+Nx+1
    I[...,1,0] = Iy*Nx+Ix+Nx+1
    I[...,1,1] = Iy*Nx+Ix+1
    I[...,1,2] = Iy*Nx+Ix

    index_buffer.unmap()

    return vertex_buffer, index_buffer, max_triangles


def update_vertices(t, vertex_buffer):
    r = (t / 10.0) % 1.0
    r = (1 - r) if r > 0.5 else r

    with vertex_buffer as V:
        (Ny, Nx, _) = V.shape
        θ = cp.linspace(0, np.pi, Nx)[None,:]
        φ = cp.linspace(0, 2*np.pi, Ny)[:,None]
        V[..., 0] = r * cp.cos(φ) * cp.sin(θ)
        V[..., 1] = r * cp.sin(φ) * cp.sin(θ)
        V[..., 2] = r * cp.cos(θ)


def main():
    if not glfw.init():
        return
    title = "CuPy Cuda/OpenGL interop example"
    window = glfw.create_window(800, 800, title, None, None)

    if not window:
        glfw.terminate()
        return

    glfw.make_context_current(window)
    glfw.swap_interval(0)

    shader = OpenGL.GL.shaders.compileProgram(
        OpenGL.GL.shaders.compileShader(VERTEX_SHADER, GL_VERTEX_SHADER),
        OpenGL.GL.shaders.compileShader(FRAGMENT_SHADER, GL_FRAGMENT_SHADER),
    )
    positionLoc = glGetAttribLocation(shader, "position")
    transformLoc = glGetUniformLocation(shader, "transform")

    Nx, Ny = 32, 32
    vertex_buffer, index_buffer, Ntriangles = setup_buffers(Nx, Ny)

    fps = 0
    nframes = 0
    last_time = glfw.get_time()

    glUseProgram(shader)
    glEnable(GL_DEPTH_TEST)

    while not glfw.window_should_close(window):
    # while 0:
        t = glfw.get_time()
        dt = t - last_time
        if dt >= 1.0:
            fps = nframes / dt
            last_time = t
            nframes = 0

        update_vertices(t, vertex_buffer)

        width, height = glfw.get_window_size(window)
        glViewport(0, 0, width, height)

        glClearColor(0.0, 0.0, 0.0, 1.0)
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)

        rot_x = pyrr.Matrix44.from_x_rotation(0)
        rot_y = pyrr.Matrix44.from_y_rotation(t)

        glUniformMatrix4fv(transformLoc, 1, GL_FALSE, rot_x * rot_y)

        glBindBuffer(GL_ARRAY_BUFFER, vertex_buffer.gl_buffer)
        glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, index_buffer.gl_buffer)
        glEnableVertexAttribArray(positionLoc)
        glVertexAttribPointer(positionLoc, 3, GL_FLOAT, GL_FALSE, 12, ctypes.c_void_p(0))
        glDrawElements(GL_TRIANGLES, int(3 * Ntriangles), GL_UNSIGNED_INT, None)

        glfw.swap_buffers(window)
        glfw.poll_events()
        glfw.set_window_title(window, f"{title} ({fps:.1f} fps)")
        nframes += 1

    index_buffer.unregister()
    vertex_buffer.unregister()

    glfw.terminate()


if __name__ == "__main__":
    # sys.exit(main())
    main()

# n.unmap()
# n.unregister()

# glBindTexture(GL_TEXTURE_3D, 2)
# # cudart.cudaGraphicsGLRegisterImage(0, GL_TEXTURE_3D, 0)
# # cudart.cudaGraphicsGLRegisterBuffer(2, 0)


# n=CudaOpenGLMappedArray(dsa.dtype, dsa.shape, 2, 0)
# n.register()
# n.map()

1 <class 'int'>
2 <class 'int'>
<cudaGraphicsResource_t 0x1aa6471d3c0>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa6471d060>
<cudaGraphicsResource_t 0x1aa647

In [4]:
glBindTexture(GL_TEXTURE_3D, 2)
# cudart.cudaGraphicsGLRegisterImage(0, GL_TEXTURE_3D, 0)
# cudart.cudaGraphicsGLRegisterBuffer(2, 0)

n=CudaOpenGLMappedArray(dsa.dtype, dsa.shape, 2, 0)
n.map()

GLError: GLError(
	err = 1282,
	description = b'nieprawid\xb3owa operacja',
	baseOperation = glBindTexture,
	cArguments = (GL_TEXTURE_3D, 2)
)

In [5]:
# cudart.cudaGraphicsMapResources(1, n._graphics_ressource, None)
# cudart.cudaGraphicsResourceGetMappedPointer(n._graphics_ressource)
# cp.ndarray(shape=(20,20,20), dtype=cp.float32, memptr=n._graphics_ressource.getPtr())
from OpenGL.GL import *
h = glGenBuffers(1)
glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, h)
print(h)

e, hg=cudart.cudaGraphicsGLRegisterBuffer(h, 0)
# e, hg= cudart.cudaGraphicsGLRegisterImage(h, GL_TEXTURE_3D, 0 )
print(e, hg)

cudart.cudaGraphicsMapResources(1, hg, None)


GLError: GLError(
	err = 1282,
	description = b'nieprawid\xb3owa operacja',
	baseOperation = glGenBuffers,
	pyArgs = (
		1,
		<object object at 0x000001AA17674D50>,
	),
	cArgs = (1, array([0], dtype=uint32)),
	cArguments = (1, array([0], dtype=uint32))
)

In [4]:
cudart.cudaGraphicsGLRegisterBuffer(72, 2)

(<cudaError_t.cudaErrorOperatingSystem: 304>, <cudaGraphicsResource_t 0x0>)

In [4]:
   

import vispy.gloo.glir
class hc(vispy.gloo.glir.GlirTexture3D):
    def create(self):
        return super().create()
    

# vispy.test()
%gui qt

"""
# def glCreateTexture_vf():
#     handle = gl.glCreateTexture()
#     print("------------------------", handle)
#     return handle
# gl.glCreateTexture = glCreateTexture_vf

# try:
#     vvff
# except NameError: 
# def isinstance_vf(a, b):
#     return (b == np.ndarray and isinstance(a, cp.ndarray)) or isinstance(a, b)
# isinstance = isinstance_vf
# def numpy_array_vf(data, **kvargs):
#     if type(data) == cp.ndarray:
#         return cp.array(data, **kvargs)
#     else:
#         return np.array(data, **kvargs)
# np.array = numpy_array_vf



# from vispy.gloo.texture import get_dtype_limits
# def convert_dtype_and_clip(data, dtype, copy=False):
#     
#     cast dtype to a new one, but first clip data to the new dtype's limits if needed
#     
#     old_min, old_max = get_dtype_limits(data.dtype)
#     new_min, new_max = get_dtype_limits(dtype)
#     if new_max >= old_max and new_min <= old_min:
#         # no need to clip
#         try:
#             return np.array(data, dtype=dtype, copy=copy)
#         except TypeError as e:
#             # might be a cupy array, which raises error with implicit conversion
#             try:
#                 import cupy as cp
#             except ImportError:
#                 raise e
#             return cp.array(data, dtype=dtype, copy=copy)
#     else:
#         # to reduce copying, we clip into a pre-generated array of the right dtype
#         new_data = np.empty_like(data, dtype=dtype)
#         # allow "unsafe" casting here as we're explicitly clipping to the
#         # range of the new dtype - this was a default before numpy 1.25
#         np.clip(data, new_min, new_max, out=new_data, casting="unsafe")
#         return new_data
# vispy.gloo.texture.convert_dtype_and_clip = convert_dtype_and_clip
"""


'\n# def glCreateTexture_vf():\n#     handle = gl.glCreateTexture()\n#     print("------------------------", handle)\n#     return handle\n# gl.glCreateTexture = glCreateTexture_vf\n\n# try:\n#     vvff\n# except NameError: \n# def isinstance_vf(a, b):\n#     return (b == np.ndarray and isinstance(a, cp.ndarray)) or isinstance(a, b)\n# isinstance = isinstance_vf\n# def numpy_array_vf(data, **kvargs):\n#     if type(data) == cp.ndarray:\n#         return cp.array(data, **kvargs)\n#     else:\n#         return np.array(data, **kvargs)\n# np.array = numpy_array_vf\n\n\n\n# from vispy.gloo.texture import get_dtype_limits\n# def convert_dtype_and_clip(data, dtype, copy=False):\n#     \n#     cast dtype to a new one, but first clip data to the new dtype\'s limits if needed\n#     \n#     old_min, old_max = get_dtype_limits(data.dtype)\n#     new_min, new_max = get_dtype_limits(dtype)\n#     if new_max >= old_max and new_min <= old_min:\n#         # no need to clip\n#         try:\n#         

In [25]:

# import vispy.app

# vispy.app.use_app("pyqt6")

fig = vispy.plot.Fig(bgcolor='k', size=(800, 800), show=False)

# vol_data = np.load(io.load_data_file('brain/mri.npz'))['data']
# vol_data = np.flipud(np.rollaxis(vol_data, 1))
vol_data = np.flip(dsa, axis=0).swapaxes(1,2)
# vol_data = cp.asarray(vol_data)
vol_data = vol_data.astype(np.float32)
e = (np.arange((563* 512* 512))/(563* 512* 512/4096)).reshape((563, 512, 512)).astype(dtype=np.float32)


clim = [32, 4096]
texture_format = "auto"  # None for CPUScaled, "auto" for GPUScaled

# v = fig[0, 0].volume(np.zeros((10,)*3, dtype=dsa.dtype), texture_format=texture_format, method='iso', clim=clim) # iso
v = fig[0].volume(vol_data, texture_format=texture_format, method='iso', clim=clim) # iso
# v2 = fig[0, 1].volume(vol_data, texture_format=texture_format, method='mip', clim=clim)
print(type(v))
"""
# vol_pw.view.camera.elevation = 30
# vol_pw.view.camera.azimuth = 30
# vol_pw.view.camera.scale_factor /= 1.5

# shape = vol_data.shape
# fig[1, 0].image(vol_data[:, :, shape[2] // 2], cmap='grays', clim=clim,
#                 fg_color=(0.5, 0.5, 0.5, 1), texture_format=texture_format)
# fig[0, 1].image(vol_data[:, shape[1] // 2, :], cmap='grays', clim=clim,
#                 fg_color=(0.5, 0.5, 0.5, 1), texture_format=texture_format)
# fig[1, 1].image(vol_data[shape[0] // 2, :, :].T, cmap='grays', clim=clim,
#                 fg_color=(0.5, 0.5, 0.5, 1), texture_format=texture_format)
"""

# if __name__ == '__main__':
# fig.on_close("Close")
# fig.native.setAttribute(QtCore.Qt.WA_DeleteOnClose)
# fig.native.


def update(i):
    print(i, end=";")
    # v.set_data(vol_data)
    # fig.show(run=True)

fig.show(run=True)

# timer = 
# vispy.app.Timer('auto', connect=update, start=True)
# timer.connect(update)
# timer.start()



<class 'vispy.scene.visuals.Volume'>


In [7]:
for i in range(10):
    time.sleep(1)
    v.set_data(vol_data)

In [25]:
for i in range(1):
    time.sleep(1)
    v.set_data(vol_data)

self._glir.command ('DATA', 1209, (0, 0, 0), (563, 512, 512, 1), dtype('float32'), 4095.0, 0.0)


cmd, id_, args CURRENT 0 (1,)
cmd, id_, args SIZE 1209 ((563, 512, 512, 1), 'luminance', 'r32f')
cmd, id_, args DATA 1209 ((0, 0, 0), array([[[[ 907.],
         [ 913.],
         [ 905.],
         ...,
         [1197.],
         [1218.],
         [1180.]],

        [[ 880.],
         [ 880.],
         [ 876.],
         ...,
         [1138.],
         [1155.],
         [1138.]],

        [[ 846.],
         [ 860.],
         [ 875.],
         ...,
         [1094.],
         [1099.],
         [1098.]],

        ...,

        [[ 888.],
         [ 916.],
         [ 928.],
         ...,
         [1081.],
         [1073.],
         [1074.]],

        [[ 878.],
         [ 895.],
         [ 905.],
         ...,
         [1086.],
         [1072.],
         [1071.]],

        [[ 885.],
         [ 885.],
         [ 888.],
         ...,
         [1091.],
         [1076.],
         [1077.]]],


       [[[ 905.],
         [ 904.],
         [ 907.],
         ...,
         [1181.],
         [1187.],
  

ERROR: Invoking <function update at 0x000002D01F06C940> repeat 16384
ERROR: Invoking <function update at 0x000002D0034329E0> repeat 8192


cmd, id_, args CURRENT 0 (1,)
cmd, id_, args CREATE 1334 ('FrameBuffer',)
cmd, id_, args CREATE 1332 ('RenderBuffer',)
cmd, id_, args SIZE 1332 ((800, 800), 'color')
cmd, id_, args ATTACH 1334 ('color', 1332)
cmd, id_, args CREATE 1333 ('RenderBuffer',)
cmd, id_, args SIZE 1333 ((800, 800), 'depth')
cmd, id_, args ATTACH 1334 ('depth', 1333)
cmd, id_, args FRAMEBUFFER 1334 (True,)
cmd, id_, args FUNC glViewport (0, 0, 800, 800)
cmd, id_, args FUNC glClearColor (0.0, 0.0, 0.0, 0.0)
cmd, id_, args FUNC glClear (17664,)
cmd, id_, args FUNC glDisable ('cull_face',)
cmd, id_, args FUNC glDisable ('depth_test',)
cmd, id_, args FUNC glDisable ('blend',)
cmd, id_, args UNIFORM 1205 ('u_enabled', 'int', array([1]))
cmd, id_, args DRAW 1205 ('triangle_fan', (0, 4), 1)
cmd, id_, args FUNC glFinish ()
cmd, id_, args CURRENT 0 (1,)
cmd, id_, args FRAMEBUFFER 1334 (False,)
cmd, id_, args FUNC glViewport (0, 0, 800, 800)
cmd, id_, args DELETE 1334 ()
cmd, id_, args DELETE 1332 ()
cmd, id_, args DELET

In [24]:
import OpenGL.GL as gl
gl.glBindTexture(gl.GL_TEXTURE_3D, 2)
p = dsa[..., None].astype(np.float32)
# u = e.astype(np.float32)
# print(type(GL_TEXTURE_3D))
# for i in range(1):
#     time.sleep(1)
gl.glTexImage3D(gl.GL_TEXTURE_3D,0,gl.GL_RED, 512,512,200, 0,gl.GL_LUMINANCE, gl.GL_FLOAT, None)
gl.glTexSubImage3D(gl.GL_TEXTURE_3D,0, 0,0,100, 512,512,100, gl.GL_LUMINANCE, gl.GL_FLOAT, p[180:280])

    # gl.glTexSubImage3D(gl.GL_TEXTURE_3D,0, 0,0,300, 512,512,100, gl.GL_LUMINANCE, gl.GL_FLOAT, p[180:280])

# GL_TEXTURE_3D 0 0 0 0 512 512 563 GL_LUMINANCE GL_FLOAT

In [23]:
x = v._texture._id
type(x), x, dsa.shape
e.shape

(563, 512, 512)

In [11]:
flags = cudart.cudaGraphicsRegisterFlags.cudaGraphicsRegisterFlagsWriteDiscard
# c = glBindBuffer(GL_ARRAY_BUFFER , 0)
# c = glGenBuffers(1)
# print(c, type(c))
gl.glBindTexture(GL_TEXTURE_3D, 2)
b = CudaOpenGLMappedArray(dsa.dtype, dsa.shape +(1,) , 2, 0) #+ (1,) #v._texture._id
# b.register()
b.map()

CUDARuntimeError: cudaErrorInvalidValue: invalid argument

In [14]:
# v._texture._glir._shared._objects
# v._texture._handle_auto_texture_format() #
# v._texture.glir.__dir__()

AttributeError: 'GPUScaledTextured3D' object has no attribute '_handle'

CUDARuntimeError: cudaErrorInvalidValue: invalid argument

In [5]:
# cp.ones((2,2)).__dlpack__()
# e = (np.arange(200**3)%256).reshape((200,)*3+(1,)).astype(dtype=np.float32)
# e

array([[[[  0.],
         [  1.],
         [  2.],
         ...,
         [197.],
         [198.],
         [199.]],

        [[200.],
         [201.],
         [202.],
         ...,
         [141.],
         [142.],
         [143.]],

        [[144.],
         [145.],
         [146.],
         ...,
         [ 85.],
         [ 86.],
         [ 87.]],

        ...,

        [[232.],
         [233.],
         [234.],
         ...,
         [173.],
         [174.],
         [175.]],

        [[176.],
         [177.],
         [178.],
         ...,
         [117.],
         [118.],
         [119.]],

        [[120.],
         [121.],
         [122.],
         ...,
         [ 61.],
         [ 62.],
         [ 63.]]],


       [[[ 64.],
         [ 65.],
         [ 66.],
         ...,
         [  5.],
         [  6.],
         [  7.]],

        [[  8.],
         [  9.],
         [ 10.],
         ...,
         [205.],
         [206.],
         [207.]],

        [[208.],
         [209.],
      

In [8]:
# v._texture._glir.command("DATA", 29, (0,0,0), np.ones((200,200,200,1)))

e = (np.arange((563* 512* 512))/(563* 512* 512/4096)).reshape((563, 512, 512,1)).astype(dtype=np.float32)[::1]
v._texture.set_data(e)
# v._texture._glir.command("DATA", 29, (0,0,0), e)
# v._texture
# fig.show(run=True)

self._glir.command ('DATA', 29, (0, 0, 0), (563, 512, 512, 1), dtype('float32'), 4096.0, 0.0)


cmd, id_, args CURRENT 0 (1,)
cmd, id_, args SIZE 29 ((563, 512, 512, 1), 'luminance', 'r32f')
cmd, id_, args DATA 29 ((0, 0, 0), array([[[[0.00000000e+00],
         [2.77531090e-05],
         [5.55062179e-05],
         ...,
         [1.41263325e-02],
         [1.41540850e-02],
         [1.41818384e-02]],

        [[1.42095918e-02],
         [1.42373443e-02],
         [1.42650977e-02],
         ...,
         [2.83359233e-02],
         [2.83636767e-02],
         [2.83914302e-02]],

        [[2.84191836e-02],
         [2.84469370e-02],
         [2.84746885e-02],
         ...,
         [4.25455160e-02],
         [4.25732695e-02],
         [4.26010229e-02]],

        ...,

        [[7.23268223e+00],
         [7.23270988e+00],
         [7.23273754e+00],
         ...,
         [7.24680853e+00],
         [7.24683619e+00],
         [7.24686384e+00]],

        [[7.24689150e+00],
         [7.24691963e+00],
         [7.24694729e+00],
         ...,
         [7.26101780e+00],
         [7.26104593e+

In [1]:
# -*- coding: utf-8 -*-
# vispy: gallery 30
# -----------------------------------------------------------------------------
# Copyright (c) Vispy Development Team. All Rights Reserved.
# Distributed under the (new) BSD License. See LICENSE.txt for more info.
# -----------------------------------------------------------------------------
"""
Display an Image
================

Simple use of SceneCanvas to display an Image.

"""

import sys

import cupy as cp

from vispy import scene
from vispy import app
# from vispy.io import load_data_file, read_png

canvas = scene.SceneCanvas(keys='interactive')
canvas.size = 800, 600
canvas.show()

# Set up a viewbox to display the image with interactive pan/zoom
view = canvas.central_widget.add_view()

# Create the image
# img_data = cp.array(read_png(load_data_file('mona_lisa/mona_lisa_sm.png')))
img_data = (cp.arange(800*600).reshape((800,600))/(800*600/256)).astype(cp.uint8)
interpolation = 'nearest'

image = scene.visuals.Image(img_data, interpolation=interpolation,
                            parent=view.scene, method='subdivide')

canvas.title = 'Spatial Filtering using %s Filter' % interpolation

# Set 2D camera (the camera will scale to the contents in the scene)
view.camera = scene.PanZoomCamera(aspect=1)
# flip y-axis to have correct aligment
view.camera.flip = (0, 1, 0)
view.camera.set_range()
view.camera.zoom(0.1, (250, 200))

# get interpolation functions from Image
names = image.interpolation_functions
names = sorted(names)
act = 17


# Implement key presses
@canvas.events.key_press.connect
def on_key_press(event):
    global act
    if event.key in ['Left', 'Right']:
        if event.key == 'Right':
            step = 1
        else:
            step = -1
        act = (act + step) % len(names)
        interpolation = names[act]
        image.interpolation = interpolation
        canvas.title = 'Spatial Filtering using %s Filter' % interpolation
        canvas.update()


if __name__ == '__main__' and sys.flags.interactive == 0:
    app.run()


RFBOutputContext()

TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.

In [4]:
from cuda import cudart

from vispy.gloo.dlpack import dlpack, util


def cuda_check(ret):
    err = ret[0]
    if err != cudart.cudaError_t.cudaSuccess:
        _, errname = cudart.cudaGetErrorName(err)
        errcode = str(int(err))
        _, errdescr = cudart.cudaGetErrorString(err)
        raise RuntimeError(errdescr.decode() + ' (' + errname.decode() + ' ' + errcode + ')')
    return ret[1:] if len(ret) != 2 else ret[1]

class CudaBuffer:
    class _Mapping:
        def __init__(self, buffer, stream):
            self.buffer = buffer
            self.stream = stream
            self.resource = cuda_check(cudart.cudaGraphicsGLRegisterBuffer(buffer._glir_object.handle, buffer._flags))
            cuda_check(cudart.cudaGraphicsMapResources(1, self.resource, self.stream))
            self.ptr, self.size = cuda_check(cudart.cudaGraphicsResourceGetMappedPointer(self.resource))
        def capsule(self):
            buffer = self.buffer
            return util.create_dlpack_capsule(self, self.ptr, buffer._device, buffer._dtype, buffer._shape, buffer._strides, buffer._byte_offset)
        def __del__(self):
            cuda_check(cudart.cudaGraphicsUnmapResources(1, self.resource, self.stream))
            cuda_check(cudart.cudaGraphicsUnregisterResource(self.resource))

    def __init__(self, glir_object, shape, dtype, strides = None, byte_offset = 0, read = False, write = True):
        self._device_type = dlpack.DLDeviceType.kDLCUDA
        self._device_id = 0 # WARNING: hardcoded to device id 0
        self._device = dlpack.DLDevice(self._device_type, self._device_id)
        self._glir_object = glir_object
        self._shape = shape
        self._dtype = dlpack.DLDataType.TYPE_MAP[str(dtype)]
        self._strides = strides
        self._byte_offset = byte_offset
        if read and write:
            self._flags = cudart.cudaGraphicsRegisterFlags.cudaGraphicsRegisterFlagsNone
        elif read:
            self._flags = cudart.cudaGraphicsRegisterFlags.cudaGraphicsRegisterFlagsReadOnly
        elif write:
            self._flags = cudart.cudaGraphicsRegisterFlags.cudaGraphicsRegisterFlagsWriteDiscard
        else:
            raise ValueError('neither read nor write set')

    def __dlpack__(self, stream=None):
        return self._Mapping(self, stream).capsule()

    def __dlpack_device__(self):
        return self._device_type, self._device_id

if __name__ == '__main__':
    import vispy.app
    import numpy as np, vispy.gloo.glir

    vertex = """
        uniform float theta;
        attribute vec4 color;
        attribute vec2 position;
        varying vec4 v_color;
        void main()
        {
            float ct = cos(theta);
            float st = sin(theta);
            float x = 0.75* (position.x*ct - position.y*st);
            float y = 0.75* (position.x*st + position.y*ct);
            gl_Position = vec4(x, y, 0.0, 1.0);
            v_color = color;
        } """

    fragment = """
        varying vec4 v_color;
        void main()
        {
            gl_FragColor = v_color;
        } """

    class Canvas(vispy.app.Canvas):
        def __init__(self):
            super().__init__(size=(512, 512), title='Rotating quad',
                             keys='interactive')
            # Build program & data
            print('Sending position buffer to vispy:')

            self.program = vispy.gloo.Program(vertex, fragment, count=4)
            self.program['color'] = [(1, 0, 0, 0), (0, 1, 0, 0),
                                     (0, 0, 1, 0), (0, 0, 0, 1)]
            self.program['position'] = [(-1, -1), (-1, +1),
                                        (+1, -1), (+1, +1)]
            self.program['theta'] = 0.0

            self.timer = vispy.app.Timer('auto', self.on_timer)
            self.clock = 0
            self.timer.start()

            vispy.gloo.set_viewport(0, 0, *self.physical_size)
            vispy.gloo.set_clear_color('white')

            np_program_buf = self.program._buffer
            np_position_buf = np_program_buf['position']
            print('position buffer =', np_position_buf)

            #self.context.glir.associate(self.program.glir)
            #self.context.glir.flush(self.context.shared.parser)
            self.show()

            byte_offset = np_position_buf.__array_interface__['data'][0] - np_program_buf.__array_interface__['data'][0]
            strides = [stride // np_position_buf.dtype.itemsize for stride in np_position_buf.strides]
            gl_handle = self.context.shared.parser.get_object(self.program['position'].base.id)

            print('Pulling position out:')
            dlpack_buf = CudaBuffer(gl_handle, np_position_buf.shape, np_position_buf.dtype, strides, byte_offset, read = True)
            import torch
            print('position torch tensor = ', torch.from_dlpack(dlpack_buf))
            import cupy
            print('position cupy array = ', cupy.from_dlpack(dlpack_buf))


        def on_draw(self, event):
            vispy.gloo.clear()
            self.program.draw('triangle_strip')

        def on_resize(self, event):
            vispy.gloo.set_viewport(0, 0, *event.physical_size)

        def on_timer(self, event):
            self.clock += 0.001 * 1000.0 / 60.
            self.program['theta'] = self.clock
            self.update()

    c = Canvas()
    c.show()
    vispy.app.run()

Sending position buffer to vispy:
position buffer = [[-1. -1.]
 [-1.  1.]
 [ 1. -1.]
 [ 1.  1.]]
Pulling position out:
position torch tensor =  tensor([[-1., -1.],
        [-1.,  1.],
        [ 1., -1.],
        [ 1.,  1.]], device='cuda:0')
position cupy array =  [[-1. -1.]
 [-1.  1.]
 [ 1. -1.]
 [ 1.  1.]]


In [1]:
import math
from vispy import app, gloo
%gui qt

In [33]:
import wx
import math
from vispy import app, gloo, scene


class Canvas(app.Canvas):

    def __init__(self, *args, **kwargs):
        app.Canvas.__init__(self, *args, **kwargs)
        self._timer = app.Timer('auto', connect=self.on_timer, start=True)
        self.tick = 0

    # def on_draw(self, event):
    #     gloo.clear(color=True)

    def on_timer(self, event):
        self.tick += 1 / 60.0
        c = abs(math.sin(self.tick))
        gloo.set_clear_color((c, c, c, 1))
        self.update()
        # print("ee", end=";")

    def on_key_press(self, event):
        if event.key.name == 'Q': self.on_close(None)

    def on_close(self, event): # broken behaviour
        print("dupa\n")
        self._timer.stop()
        self.close()
# if __name__ == '__main__':
#     
#     
#
axis = scene.visuals.XYZAxis()
c = Canvas(keys='interactive', always_on_top=False)

# @c.events.close.connect


c.show()
app.run()

# c._timer.stop()
# c.close()

In [15]:
print(type(c.native))
q = c #.context #= c #.app.native.windows_events
print(q)
sorted(q.__dir__())

<class 'vispy.app.backends._qt.CanvasBackendDesktop'>
<Canvas (PyQt5) at 0x17e65667550>


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_app',
 '_autoswap',
 '_backend',
 '_backend_kwargs',
 '_basetime',
 '_closed',
 '_context',
 '_dpi',
 '_fps',
 '_fps_callback',
 '_fps_window',
 '_frame_count',
 '_ipython_display_',
 '_keys_check',
 '_px_scale',
 '_repr_mimebundle_',
 '_set_keys',
 '_timer',
 '_title',
 '_update_fps',
 'app',
 'close',
 'connect',
 'context',
 'create_native',
 'dpi',
 'events',
 'fps',
 'fullscreen',
 'measure_fps',
 'native',
 'on_close',
 'on_key_press',
 'on_timer',
 'physical_size',
 'pixel_scale',
 'position',
 'render',
 'set_current',
 'show',
 'size',
 'swap_buffers',
 'tick',
 'title',
 'update']

In [57]:
import vispy.scene
import vispy.plot

# vispy.scene.canvas.central_widget

fig = vispy.plot.Fig(bgcolor='k', size=(800, 800), show=False)

# v = fig[0, 0].volume(np.zeros((10,)*3, dtype=dsa.dtype), texture_format=texture_format, method='iso', clim=clim) # iso

fig.show(run=True)

In [62]:
# fig[0].camera.#transforms.get_transform().simplified#.__dir__()

0.0

In [38]:

import numpy as np


from vispy import app, scene, io, visuals
from vispy.visuals.transforms import STTransform

# Read volume
vol = dsa

canvas = scene.SceneCanvas(keys='interactive', size=(800, 600), show=True)
view = canvas.central_widget.add_view()
volume = visuals.volume(vol, parent=view.scene)
volume.transform = scene.STTransform(translate=(64, 64, 0))

cam = scene.cameras.TurntableCamera(parent=view.scene, fov=60)
view.camera = cam

# Create an XYZAxis visual
axis = scene.visuals.XYZAxis(parent=view)
s = STTransform(translate=(50, 50), scale=(50, 50, 50, 1))
affine = s.as_matrix()
axis.transform = affine

app.run()

TypeError: 'module' object is not callable

In [74]:
import vispy.plot
import vispy.app

fig = vispy.plot.Fig(bgcolor='k', size=(800, 800), show=True)
volume = fig[0].volume(np.flip(vol, axis=0), texture_format='auto', method='mip') # iso

canvas = volume.canvas
timer = app.Timer()

@canvas.events.key_press.connect
def on_key_press(event):
    if event.key.name == 'Q': 
        timer.stop()
        canvas.close()

def on_timer(event):
    print(canvas)
timer.connect(on_timer)

timer.start()
app.run()


dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
dupa
